In [1]:
from keras.models import Sequential
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.


In [5]:
#import data
features = pd.read_csv("C:\\Users\\kyle.becker\\Desktop\\Walmart LSTM\\features.csv")
stores = pd.read_csv("C:\\Users\\kyle.becker\\Desktop\\Walmart LSTM\\stores.csv")
train = pd.read_csv("C:\\Users\\kyle.becker\\Desktop\\Walmart LSTM\\train.csv")
test = pd.read_csv("C:\\Users\\kyle.becker\\Desktop\\Walmart LSTM\\test.csv")

In [7]:
#join dataframes together to get training set
train_features = pd.merge(left=train, right=features, on = ['Store', 'Date'])
train_features_stores = pd.merge(left=train_features,right=stores, on = ['Store'])

#Check rows before and after
print(train.shape)
print(train_features_stores.shape) 

#rename 
train_df = train_features_stores

(421570, 5)
(421570, 17)


In [8]:
#join dataframes together for test set
test_features = pd.merge(left=test,right=features, on = ['Store', 'Date'])
test_features_stores = pd.merge(left=test_features, right=stores, on = ['Store'])

#check rows before and after
print(test.shape)
print(test_features_stores.shape)

test_df = test_features_stores

(115064, 4)
(115064, 16)


In [9]:
#see column that does not exist in test data set
missing_column = [train_df.columns[i] for i in range(0,len(train_df.columns)) if train_df.columns[i] not in test_df.columns]
print(missing_column)

#column is missing weekly sales because the results must be submitted to be scored by the contest

['Weekly_Sales']


In [10]:
#order by date and store
train_df_sorted = train_df.sort_values(by=['Store', 'Dept', 'Date'])
#shift values so prior sales can be used to predict future
train_df_sorted['Weekly_Sales'] = train_df_sorted.groupby(['Store', 'Dept'])['Weekly_Sales'].shift(1)
train_df_sorted['Temperature'] = train_df_sorted.groupby(['Store', 'Dept'])['Temperature'].shift(1)
train_df_sorted['Fuel_Price'] = train_df_sorted.groupby(['Store', 'Dept'])['Fuel_Price'].shift(1)
train_df_sorted['CPI'] = train_df_sorted.groupby(['Store', 'Dept'])['CPI'].shift(1)
train_df_sorted['Unemployment'] = train_df_sorted.groupby(['Store', 'Dept'])['Unemployment'].shift(1)

In [11]:
#filter to remove na
train_df_sorted = train_df_sorted[train_df_sorted['Weekly_Sales'].notnull()]

In [13]:
#select key columns and rename duplicate column
train_for_model = train_df_sorted.loc[:,['Date','Store','Dept','IsHoliday_x', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2',
                                       'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 
                                        'Weekly_Sales']].rename(columns={'IsHoliday_x':'IsHoliday'})

In [14]:
#fill in 0
train_for_model = train_for_model.fillna(0)

In [16]:
#convert IsHoliday to binary
train_for_model = pd.get_dummies(train_for_model, columns=['IsHoliday'])

In [18]:
#split data to normalize
train_for_model_identifiers = train_for_model.loc[:,['Date', 'Store', 'Dept']].reset_index().drop(columns='index')
train_for_model_features = train_for_model.drop(columns = ['Date', 'Store', 'Dept']).reset_index().drop(columns='index')
print(train_for_model_features.columns)

Index(['Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3',
       'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Weekly_Sales',
       'IsHoliday_False', 'IsHoliday_True'],
      dtype='object')


In [19]:
#normalize data
normalize = MinMaxScaler(feature_range=(0,1))
features_scaled = normalize.fit_transform(train_for_model_features)

#normalize y
output_scaled = normalize.fit_transform(train_for_model_features['Weekly_Sales'].values.reshape(-1,1))

df_features_scaled = pd.DataFrame(features_scaled, columns = train_for_model_features.columns).reset_index()
#Y_scaled = normalize.fit_transform(Y)

C:\Users\kyle.becker\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [ ]:
#Join back 
df_train = pd.concat([train_for_model_identifiers, df_features_scaled], axis=1, sort=False)
df_train = df_train.drop(columns=['index'])

In [24]:
def create_lstm_groupings(df,lookback):
    #blank list to hold lagged sales for lstm
    groupings_df = []
    #sort data by store dept and date to find
    #previous sales lag
    df = df.sort_values(by=['Store', 'Dept', 'Date'])
    #convert df to list
    X_list = df.values.tolist()
    for row in range(2,len(X_list)):
        current_row = X_list[row]
        grouped_list = [current_row]
        for lookback in range(1,lookback+1):
            #find sales records that occur
            #specified lags before record
            previous_row = [X_list[row-lookback]]
            
            if previous_row[0][1]==grouped_list[0][1] and previous_row[0][2]==grouped_list[0][2]:
                #if the lagged records have the same store and department add to list
                grouped_list = grouped_list+previous_row
        if len(grouped_list)==lookback:
            #if user defined lag of records is found append to list
            groupings_df.append(grouped_list)
    
    return groupings_df
            

In [25]:
#run user defined function to create groups of lagged sales
X_groupings  = create_lstm_groupings(df_train,2)

In [27]:
#remove identifying information (store, department) to pass to model
def remove_identifiers(X_list):
    X_scaled = []
    Y_scaled = []
    for i in range(0,len(X_list)):
        record = []
        Y_actual = X_list[i][0][12]
        Y_scaled.append(Y_actual)
        for j in range(0,len(X_list[i])):
            record = record+[X_list[i][j][3:12]+X_list[i][j][13:15]]
        X_scaled.append(record)
    return X_scaled, Y_scaled

In [28]:
#run user defined function to remove identifying information
X_scaled, Y_scaled = remove_identifiers(X_groupings)

In [30]:
#convert list to array as this is the format lstm 
#takes data in
def add_results_to_array(X_scaled):
    X_scaled_array = np.empty((0,3,11), float)
    for i in range(0,len(X_scaled)):
        if  i%10000==0:
            print(i)
        row = X_scaled[i]
        X_scaled_array = np.append(X_scaled_array, [row], axis=0)
    return X_scaled_array

In [ ]:
#run user defined function to convert list to array
array_lstm = add_results_to_array(X_scaled)

In [35]:
#save results to pickle
import pickle
output = open('C:\\Users\\kyle.becker\\Desktop\\Walmart LSTM\\array_lstm.pkl', 'wb')
pickle.dump(array_lstm, output)

In [ ]:
#read in array
import pickle
infile = open('C:\\Users\\kyle.becker\\Desktop\\Walmart LSTM\\array_lstm.pkl','rb')
array_lstm = pickle.load(infile)
array_lstm.shape

In [5]:
#split data set variables into test and train
import random

train_sample = np.random.choice(array_lstm.shape[0], 300000, replace=False)
list_train_sample = list(train_sample)

all_sample =[i for i in range(0,array_lstm.shape[0])]

test_sample = [i for i in range(0,array_lstm.shape[0]) if i not in train_sample]

train_X = array_lstm[train_sample, :]
test_X = array_lstm[test_sample, :]

In [ ]:
#split output variable into train and test
Y = np.array([Y_scaled]).reshape(-1,1)

train_Y = Y[train_sample,:]
test_Y = Y[test_sample,:]

In [506]:
#create lstm model
model = Sequential()
model.add(LSTM(units=10, activation='tanh', input_shape=(3,11)))
model.add(Dense(units=1))
#use adam optimization and mean squared error to measure results
model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
#fit model
model.fit(train_X,train_Y, epochs=10, batch_size=1)

In [518]:
#reverse normalization
Y_train_pred = model.predict(train_X)
Y_train_pred = normalize.inverse_transform(Y_train_pred)
#calculate training error
training_error = mean_squared_error(train_Y,Y_train_pred)
#print training error
print(training_error)

#reverse normalization
Y_test_pred = model.predict(test_X)
Y_test_pred = normalize.inverse_transform(Y_test_pred)
#calculate test error
testing_error = mean_squared_error(test_Y,Y_test_pred)
#print test error
print(testing_error)

394260776.36218774
394242666.32123613


In [ ]:
#model is experiencing large bias need to increase the number of epochs to decrease bias
# and increase the learning rate